# Exploratory data analysis

The objective of this notebook is to present <a href="https://en.wikipedia.org/wiki/Exploratory_data_analysis">**exploratory data analysis**</a> (EDA), which encourage statisticians to explore the data, and possibly formulate hypotheses that could lead to new data collection and experiments.

The main distinction from EDA to our previous initial analysis is that while in the initial analysis we were concerned with the quality of the data and its suitability for analysis, during EDA we will focus on answering questions directly related to our research objective: 

**Example:** Which are the main attributes with the most impact on the car price?

#### Required libraries
* <a href = "https://pandas.pydata.org/"><code>Pandas</code></a>
* <a href = "https://www.scipy.org/"><code>Scipy</code></a> 
* <a href = "https://numpy.org/"><code>Numpy</code></a> 
* <a href = "https://numpy.org/"><code>Scikit-learn</code></a>
* <a href = "https://matplotlib.org/"><code>Matplotlib</code></a>
* <a href = "https://seaborn.pydata.org/"><code>Seaborn</code></a>

#### Table of contents
<ol>
    <li><a href="data_types">Data types</a></li>
    <li><a href="univariate_descriptive_analysis">Univariate descriptive analysis</a><br>
        2.1. <a href="measures_central_tendency">Measures of central tendency</a><br>
        2.2. <a href="measures_variability">Measures of variability</a><br>
        2.3. <a href="density_estimation">Density estimation</a>
    </li>
    <li><a href="bivariate_descriptive_analysis">Bivariate descriptive analysis</a><br>
        3.1. <a href="analysis_via_visualization">Analysis via visualization</a><br>
        3.2. <a href="correlation">Correlation</a><br>
        3.3. <a href="mutual_information">Mutual information</a><br>
    </li>
</ol>

----

<h2 id="attribute_types">1 - Data types</h2>

<img src="images/attribute_types.jpg" width="700">

#### `Pandas` data types

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("../Data/Automobile_nomissing.csv")
df.head(10)
#df.select_dtypes(exclude="object").head(10)

#### <code>Pandas</code> categorical data
All values of categorical data are either in categories or np.nan. Order is defined by the order of categories, not lexical order of the values. Internally, the data structure consists of a categories array and an integer array of codes which point to the real value in the categories array.

The <code>categorical</code> data type is useful in the following cases:

* A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory (its memory usage is proportional to the number of categories plus the length of the data. In contrast, an object dtype is a constant times the length of the data).

In [ ]:
s = pd.Series(['foo', 'bar'] * 100)
print(s.nbytes)
print(s.astype('category').nbytes)

* The lexical order of a variable is not the same as the logical order (i.e., *low*, *medium*, *high*). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order.

In [ ]:
s_array = ["low", "medium", "medium", "low", "high"]

# Categorical with lexical order 
s = pd.Series(pd.Categorical(s_array, ordered=True))
s.sort_values(inplace=True)
print(s)
print("Min: " + str(s.min()))
print("Max: " + str(s.max()))

In [ ]:
# Categorical with specific order
s = pd.Series(pd.Categorical(s_array, categories = ["low", "medium", "high"], ordered=True))
s.sort_values(inplace=True)
print(s)
print("Min: " + str(s.min()))
print("Max: " + str(s.max()))

* As a signal to other Python libraries that this column should be treated as a categorical variable (to give more information or for specific methods).
    
    * <a href="https://en.wikipedia.org/wiki/Ordinal_regression">**Ordinal regression**</a>

<h2 id="descriptive_univariate_analysis">2 - Descriptive univariate analysis</h2>

Univariate analysis separately describes each attribute in a dataset. It can be applied to both discrete and continuous attributes with some differences.

<h3 id="measures_central_tendency">2.1 - Measures of central tendency</h3>

Central tendency is one of the most common numerical descriptive measures. It's used to estimate the central location of the univariate data by the calculation of **mean**, **median** and **mode**.

* The <a href="https://en.wikipedia.org/wiki/Mean">**arithmetic mean**</a> is the central value of a discrete set of numbers: specifically, the sum of the values divided by the number of values. If the data set were based on a series of observations obtained by sampling from a statistical population, the arithmetic mean of an attribute $X$ is the sample mean (denoted $\displaystyle {\bar{x}})$ to distinguish it from the mean of the underlying distribution, the population mean $\mu_{x}$. 

    * It can be estimated with:
        * numerical attributes.

<img src="images/mean_diagram.png" width="300">

In [ ]:
df["price"].mean()

* The <a href="https://en.wikipedia.org/wiki/Median">**median**</a> is the value separating the higher half from the lower half of a data sample (a population or a probability distribution). For a data set, it may be thought of as the "middle" value. For example, in the data set [1, 3, 3, 6, 7, 8, 9], the median is 6, the fourth largest, and also the fourth smallest, number in the sample. For a continuous probability distribution, the median is the value such that a number is equally likely to fall above or below it. 

    * It can be estimated with:
        * numerical attributes.
        * ordinal attributes.

<img src="images/median_diagram.png" width="300">

In [ ]:
df["price"].median()

* The <a href="https://en.wikipedia.org/wiki/https://en.wikipedia.org/wiki/Mode_(statistics)">**mode**</a> of a set of data values is the value that appears most often. In probability distributions is often considered to be any value x at which its probability density function has a **locally maximum value**, so any peak is a mode.

    * It can be estimated with:
        * numerical attributes.
        * ordinal attributes.
        * nominal attributes.

<img src="images/mode_diagram.png" width="300">

In [ ]:
df["price"].mode()

There are multiple modes in the car price attribute. 

In order to estimate the mode of the underlying distribution, the usual practice is to **discretize** the data by assigning frequency values to intervals of equal distance (as for making a histogram) effectively replacing the values by the **midpoints of the intervals** they are assigned to. 

<h3 id="measures_variability">2.2 - Measures of variability</h3>

<span style="color:red">**Note:**</span> There are no measures of variability for discrete or categorical data.

A measure of  <a href="https://en.wikipedia.org/wiki/Statistical_dispersion">**variability / dispersion**</a> (deviation from the mean) of a univariate data set can reveal the shape of a univariate data distribution more sufficiently. It will provide some information about the variation among data values. The measures of variability together with the measures of central tendency give a better picture of the data than the measures of central tendency alone. The three most frequently used are **range**, **inter-quartile range**, **variance** and **standard deviation**. 

* The <a href="https://en.wikipedia.org/wiki/Range_(statistics)">**range**</a> is the difference between the largest and smallest values in a set of values. For example, consider the following numbers: 1, 1, 3, 5, 7, 8, 10, 11, 11, 15. Its range would be 15 - 1 or 14.

* The <a href="https://en.wikipedia.org/wiki/Interquartile_range">**inter-quartile range**</a> (IQR) is a measure of variability based on quartiles, which are cut points that divide a dataset into four more or less equal parts (**in density terms**), or quarters. The IQR is the first quartile subtracted from the third quartile. It is usually graphically represented with a box plot.

    * The second quartile is the median.
    * min: $Q_{1} - 1.5*\text{IQR}$
    * max: $Q_{3} + 1.5*\text{IQR}$

<img src="images/quartiles.jpg" width="400">



In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
ax = sns.boxplot(x=df["price"])

* The <a href="">**variance**</a> measures how far a set of values are spread out from their average value. It is defined by the following formula: $$ \displaystyle \sigma^{2}(\mathbf{x}) = \frac{\sum_{i} x_{i} - \mu_{x}}{m}$$

In [ ]:
df["price"].var()

* The <a href="">**standard deviation**</a> is the square root of the variance.

In [ ]:
df["price"].std()

#### All together

We can generate a set of descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution (excluding missing values) using the <code>describe()</code> method.

In [ ]:
df["price"].describe()

<h3 id="density_estimation">2.3 - Density estimation</h3>

Density estimation is the construction of an **estimate**, based on observed data, **of an unobservable underlying probability density function**. The unobservable density function is thought of as the density according to which a large population is distributed; the data are usually thought of as a random sample from that population.

Therefore, a density estimator is an algorithm which takes a $D$-dimensional dataset and produces an estimate of the $D$-dimensional probability distribution which data is drawn from. Of course for univariate analysis we will focus on one-dimensional datasets (only one column).

#### Continuous attributes

There are a variaty of approaches. Two of the most used ones are the **histogram** and the **kernel density estimation** (KDE) plot.

##### **Histogram**

For one dimensional data, you are probably already familiar with one simple density estimator, the histogram, whose construction follow two steps: 
1. Divide the entire range of values into a series of intervals (bins).
2. Count how many values fall into each interval. The intervals must be consecutive, adjacent and non-overlapping. In addition they are usually of equal size, but it is not required.

Histograms give a rough sense of the density of the underlying distribution of the data, and often for density estimation: estimating the probability density function of the underlying variable. The total area of a histogram used for probability density is always normalized to 1.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

x = df["price"]

sns.distplot(x, kde=False, bins = 20, rug=True);
plt.show()

If we increase the number of bins, the result looks a bit messy, but is a much more robust reflection of the actual data characteristics than is the standard histogram. Still, the rough edges are not aesthetically pleasing, nor are they reflective of any true properties of the data. In order to smooth them out, <span style="color:blue">we might decide to replace the blocks at each location with a smooth function, like a **Gaussian**</span>.

##### **Kernel density estimation (KDE) plot**

Kernel density estimation is a non-parametric method to estimate the probability density function of a random variable. It has the following parameters:
* The <a href="https://en.wikipedia.org/wiki/Kernel_(statistics)">**kernel**</a>. It specifies the shape of the distribution placed at **each point**.
* The <a href="https://en.wikipedia.org/wiki/Kernel_density_estimation#Bandwidth_selection">**kernel bandwith**</a>. It controls the size of the kernel at **each point**.

<img src="images/kernel_types.png" width="500">

<span style="color:red">**Note:**</span> Non-parametric models differ from parametric models in that the model structure is not specified a priori but is instead determined from data. The term non-parametric is not meant to imply that such models completely lack parameters but that the number and nature of the parameters are flexible and not fixed in advance.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

x = df["price"]

sns.kdeplot(x, ax=ax1)
sns.rugplot(x, linewidth=1);
plt.show()

Drawing a KDE is more computationally expensive than drawing a histogram. What happens is that each observation is first replaced with a normal (Gaussian) curve centered at that value:

In [ ]:
from scipy import stats

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

x = df["price"]
bandwidth = x.std() * x.size ** (-1 / 5.)
support = np.linspace(x.min(), x.max(), 201)

kernels = []
for x_i in x:
    kernel = stats.norm(x_i, bandwidth).pdf(support)
    kernels.append(kernel)
    plt.plot(support, kernel, color="r")

sns.rugplot(x, linewidth=1);

<span style="color:red">**Note:**</span> The KDE is not only useful for descriptive purposes. It can be also used for <a href="https://en.wikipedia.org/wiki/Regression_analysis">**regression**</a> or <a href="https://en.wikipedia.org/wiki/Statistical_classification">**classification**</a> (as a better one-dimensional estimator that, for example, a gaussian). For those cases, we could use the <code>sklearn.neighbors.KernelDensity</code> estimator, which is able to handle KDE in multiple dimensions and allows to use different kinds of kernels (not only Gaussian).

##### **Histogram and KDE combined**

We can combine both diagrams to give a better idea of the underlying distribution.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

x = df["price"]

sns.distplot(x, bins = 20, rug=True);

plt.show()

### Discrete/Categorical attributes

In the case of discrete/categorical variables there is only one way to represent its probability distribution, the histogram, where each category is represented by an "interval"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

sns.countplot(x="num-of-doors", data=df)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))
sns.countplot(x="drive-wheels", data=df)

<h2 id="bivariate_descriptive_analysis">3 - Bivariate descriptive analysis</h2>

Bivariate descriptive analysis is used to describe the relationship between a pair of variables.

<h3 id="analysis_via_visualization">3.1 - Analysis via visualization</h3>

In order to understand the relationships between a couple of variables we can use plots, whose form will change depending on the variables' types.

#### Numerical & Numerical

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

sns.scatterplot(x=df["highway-mpg"], y=df["price"])
#sns.regplot(x=df["highway-mpg"], y=df["price"])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 4))

sns.regplot(x=df["engine-size"], y=df["price"])

plt.show()

More information about plots between numerical variables with Seaborn can be found <a href="https://seaborn.pydata.org/tutorial/regression.html">**in the documentation**</a>

#### Categorical & Numerical

There are several available plots for visualizing changes in a numerical variable conditioned on a categorical one. We can find them using the <code>catplot</code> function, which generates a categorical scatterplot (one scatterplot per category).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="price", y="make", data=df);

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="price", y="make", hue="num-of-doors", data=df, jitter=False); # we can change the width of the points via the jitter attribute

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="num-of-doors", y="price", kind = "box", data=df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="num-of-doors", y="price", kind="violin", data=df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="drive-wheels", y="price", hue="num-of-doors", kind="violin", data=df); 

# The split attribute allow us to split each violin in two, reducing its size, but loses information on the boxplot part
#sns.catplot(x="drive-wheels", y="price", hue="num-of-doors", kind="violin", split=True, data=df); 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="drive-wheels", y="price", hue="num-of-doors", kind="box", data=df)

* The **violin plot** shows a better representation of the underlying probability distribution.
* The **box plot** shows the existance of outliers better than the violin plot.

#### Categorical & Categorical

We get that "line" on the 4 wheels case with two doors because there is only one instance and the distribution then follows a <a href="https://en.wikipedia.org/wiki/Dirac_delta_function">**Dirac function**</a> (no variance).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.catplot(x="drive-wheels", hue="num-of-doors", kind="count", data=df)

<h3 id="analysis_via_visualization">3.1 - Correlation</h3>

Correlation is a statical measure of dependence. In the ML literature, its usually referred to the Pearson's correlation coefficient $\rho$, which measures linear dependence between two variables. It has a value between $+1$ and $−1$, where $1$ is total positive linear correlation, $0$ is no linear correlation, and $−1$ is total negative linear correlation. It is a symmetric measure. Given a pair of random variables $\displaystyle (X,Y)$, their pearson correlation $\rho_{X,Y}$ is

$$ \rho_{X,Y} = \frac{cov(X,Y)}{\sigma_{X} \sigma_{Y}},$$ 

where the estimation of $cov(X,Y)$ is done via the following formula: 

$$ cov(X,Y) = \frac{1}{m-1} \sum_{i}^{m} (x_{i} - \bar{x})(y_{i} - \bar{y})$$

#### Properties

* Symmetry. $\rho_{X,Y} = \rho_{Y,X}$
* Normalized in $[-1,1]$.

#### Correlation and independence

If the variables are independent, Pearson's correlation coefficient is 0, but the converse is not true because the correlation coefficient detects only **linear** dependencies between two variables.

<img src="images/correlation_independence.png" width="600">

In [ ]:
print(df[["highway-mpg", "price"]].corr())
print(df[["engine-size", "price"]].corr())

#### Correlation matrix

<span style="color:red">**Note:** <code>df.corr()</code> automatically excludes non-numerical columns from the estimation</span>

In [ ]:
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

<h3 id="mutual_information">3.2 - Mutual information</h3>

<img src="images/correlation_examples.png" width="600">

One important caveat of the correlation coefficient is its inability to consider nonlinear dependence relationships. For this reason we can complement it by estimating the bivariate <a href="https://en.wikipedia.org/wiki/Mutual_information">**Mutual information**</a>.

#### Example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression, mutual_info_regression

np.random.seed(0)
X = np.random.rand(1000, 3)
y = X[:, 0] + np.sin(6 * np.pi * X[:, 1]) + 0.1 * np.random.randn(1000)

f_test, _ = f_regression(X, y)
f_test /= np.max(f_test)

mi = mutual_info_regression(X, y)
mi /= np.max(mi)

plt.figure(figsize=(15, 5))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.scatter(X[:, i], y, edgecolor='black', s=20)
    plt.xlabel("$x_{}$".format(i + 1), fontsize=14)
    if i == 0:
        plt.ylabel("$y$", fontsize=14)
    plt.title("F-test={:.2f}, MI={:.2f}".format(f_test[i], mi[i]),
              fontsize=16)
plt.show()

In [ ]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif

mutual_info_regression(df[["highway-mpg"]], df["price"])

In [ ]:
mutual_info_classif(df[["price"]], df["num-of-doors"])
#sns.catplot(x="num-of-doors", y="price", kind = "violin", data=df) # We can graphically see why its MI is so low...

----
**Exercise: Estimate the MI matrix of the numerical variables in <code>df</code> and show it in a heatmap**

* Use <code>sklearn.feature_selection.mutual_info_regression</code> method for the bivariate estimation.
* Use previous heatmap code but with your mi_matrix instead of the correlation_matrix generated by <code>Pandas</code>.

Extra:
* We dont need to estimate the full matrix, only the lower or upper triangular matrices because MI is symmetric: $I(X,Y) = I(Y,X)$

(double click for solution)

<!--

mi_df = df.select_dtypes(exclude="object")
mi_df = mi_df.select_dtypes(exclude="category")

############################################
# Matrix estimation

n = len(mi_df.columns)
mi_matrix = np.zeros((n, n))

for i in range(0, n):
    for j in range(i+1, n):
        # Note: Invert j and i so the lower matrix is showed instead of the upper triangular
        # Note: We have to use reshape because sklearn expects a 2D array
        mi_matrix[j,i] = mutual_info_regression(mi_df.iloc[:,0].values.reshape(-1,1), mi_df.iloc[:,1]) 

############################################
# Dataframe adjustments

mi_matrix_df = pd.DataFrame(mi_matrix)
mi_matrix_df.columns = mi_df.columns # Name the columns
mi_matrix_df.index = mi_df.columns   # Name the rows

#############################################
# Heatmap code

# Generate a mask for the upper triangle
mask = np.zeros_like(mi_matrix_df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(mi_matrix_df, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
-->

----

In [ ]:
mi_df = df.select_dtypes(exclude="object")
mi_df = mi_df.select_dtypes(exclude="category")

#### Write your code under here


## References

#### Books
1. Bishop, C. M. (2006). "Pattern recognition and machine learning". Springer.
2. Murphy, K. P. (2012). "Machine Learning: A probabilistic perspective". MIT Press.
3. Stockburger D. W. (2016). "Multivariate statistics: concepts, models and applications".

#### Links
1. <a href="https://www.coursera.org/learn/data-analysis-with-python/">Santarcangelo, J. (2019). "Data analysis with Python".</a>